In [1]:
import numpy as np
import pandas as pd
import random
from scipy import sparse
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
full_df = pd.read_csv('spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False)
full_df.head()

/tmp/ipykernel_27892/2588061196.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  full_df = pd.read_csv('spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False)
/tmp/ipykernel_27892/2588061196.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  full_df = pd.read_csv('spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False)


,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


In [3]:
full_df.shape

(12891680, 4)

In [4]:
# clean up column names
full_df.columns = full_df.columns.str.replace('"', '')
full_df.columns = full_df.columns.str.replace('name', '')
full_df.columns = full_df.columns.str.replace(' ', '')
full_df.columns

Index(['user_id', 'artist', 'track', 'playlist'], dtype='object')

In [5]:
def create_dict_by_col(data, col):
    col_id_new = dict()
    i = 0
    for id in data[col].values:
        if id not in col_id_new:
            col_id_new[id] = i
            i += 1
    data[col] = data[col].apply(lambda x: col_id_new[x]).astype(int)

In [6]:
create_dict_by_col(full_df, "user_id")

In [7]:
users = full_df[full_df["user_id"] < 500]

In [8]:
def unique_elems(data, col):
    cols_set = set()
    for elem in data[col].values:
        cols_set.add(elem)
    return cols_set

In [9]:
artists = unique_elems(users, "artist")
tracks = unique_elems(users, "track")
playlists = unique_elems(users, "playlist")

In [10]:
df_playlist = full_df[(full_df["user_id"] < 500) & (full_df["artist"].isin(artists)) & (full_df["track"].isin(tracks)) & (full_df["playlist"].isin(playlists))]

In [11]:
df_playlist

,user_id,artist,track,playlist
0,0,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,0,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,0,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,0,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,0,Elvis Costello,Alison,HARD ROCK 2010
...,...,...,...,...
373278,499,Katy Perry,Teenage Dream,Pop Rocks - Naughty Pleasures for Dirty Minds
373279,499,Lady Gaga,The Edge Of Glory,Pop Rocks - Naughty Pleasures for Dirty Minds
373280,499,Michael Jackson,Thriller - Single Version,Pop Rocks - Naughty Pleasures for Dirty Minds
373281,499,Britney Spears,Toxic,Pop Rocks - Naughty Pleasures for Dirty Minds


In [12]:
df_playlist.dropna()

,user_id,artist,track,playlist
0,0,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,0,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,0,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,0,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,0,Elvis Costello,Alison,HARD ROCK 2010
...,...,...,...,...
373278,499,Katy Perry,Teenage Dream,Pop Rocks - Naughty Pleasures for Dirty Minds
373279,499,Lady Gaga,The Edge Of Glory,Pop Rocks - Naughty Pleasures for Dirty Minds
373280,499,Michael Jackson,Thriller - Single Version,Pop Rocks - Naughty Pleasures for Dirty Minds
373281,499,Britney Spears,Toxic,Pop Rocks - Naughty Pleasures for Dirty Minds


In [13]:
df_playlist.describe()

,user_id
count,373283.000000
mean,242.242371
std,142.901252
min,0.000000
25%,132.000000
50%,223.000000
75%,378.000000
max,499.000000


In [14]:
df_playlist.isnull().values.any()

True

In [15]:
df_playlist.isnull().sum()

user_id       0
artist      805
track         3
playlist     18
dtype: int64

In [16]:
for column in df_playlist.columns:
    df_playlist[column].replace('', np.nan, inplace=True)
    df_playlist.dropna(subset=[column], inplace=True)

/tmp/ipykernel_27892/1051766524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_playlist[column].replace('', np.nan, inplace=True)
/tmp/ipykernel_27892/1051766524.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_playlist.dropna(subset=[column], inplace=True)
/tmp/ipykernel_27892/1051766524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_playlist[column].replace('', np.nan, inplace=True)
/tmp/ipykernel_27892/10517665

In [17]:
df_playlist.isnull().sum()

user_id     0
artist      0
track       0
playlist    0
dtype: int64

In [18]:
df_playlist[df_playlist.duplicated()]

,user_id,artist,track,playlist


In [19]:
df_playlist = df_playlist.drop_duplicates(keep='first')

In [20]:
df_playlist["artist"].value_counts().head(10)

Coldplay                 1363
Daft Punk                1250
Johnny Cash              1244
Arctic Monkeys           1217
The Rolling Stones       1193
Red Hot Chili Peppers    1174
Foo Fighters             1130
JAY Z                    1070
Radiohead                1042
Pearl Jam                 999
Name: artist, dtype: int64

In [21]:
df_playlist[["track", "artist"]].value_counts().head(10)

track                                                     artist             
Little Talks                                              Of Monsters and Men    93
Pompeii                                                   Bastille               83
Midnight City                                             M83                    82
Radioactive                                               Imagine Dragons        75
Blurred Lines                                             Robin Thicke           72
Rather Be (feat. Jess Glynne)                             Clean Bandit           71
Get Lucky - Radio Edit                                    Daft Punk              70
Mr. Brightside                                            The Killers            70
We Are Young (feat. Janelle Monáe) - feat. Janelle Monae  Fun.                   69
Do I Wanna Know?                                          Arctic Monkeys         69
dtype: int64

In [22]:
# user-playlist's tracks number
df_playlist.groupby(["user_id", "playlist"], as_index=False)['track'].nunique()

,user_id,playlist,track
0,0,HARD ROCK 2010,67
1,0,IOW 2012,37
2,1,2080,10
3,1,C418,34
4,1,Chill out,140
...,...,...,...
6992,499,2011 CD,18
6993,499,2013 in Music,18
6994,499,6 Music Festival - Drowned in Sound,20
6995,499,Girls Aloud - Always Allowed,19


In [23]:
avg_count = df_playlist.groupby(["user_id", "playlist"], as_index=False)['track'].nunique().mean().round()
print(f"The average track count per playlist is {avg_count[0]}")

The average track count per playlist is 242.0


/tmp/ipykernel_27892/4049129021.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  avg_count = df_playlist.groupby(["user_id", "playlist"], as_index=False)['track'].nunique().mean().round()


In [24]:
# longest playlist
df_playlist.groupby(["user_id", "playlist"], as_index=False)['track'].nunique().sort_values(by=['track'], ascending=False).head(10)

,user_id,playlist,track
39,5,Everything at once,7892
2720,188,Spotify Library,7645
798,53,Strane,6815
2719,188,Rich's iPhone,4603
3032,206,Rock,4209
3028,206,Punk,3957
2566,177,Starred,3891
6269,442,Definitely not greek,3123
4113,273,Starred,2718
5706,406,best of radio paradise,2690


In [25]:
df_playlist[df_playlist["playlist"] != 'Starred'].groupby(["user_id", "playlist"], as_index=False)['track']\
        .nunique().sort_values(by=['track'], ascending=False).head(10)

,user_id,playlist,track
37,5,Everything at once,7892
2651,188,Spotify Library,7645
776,53,Strane,6815
2650,188,Rich's iPhone,4603
2958,206,Rock,4209
2954,206,Punk,3957
6114,442,Definitely not greek,3123
5563,406,best of radio paradise,2690
769,53,Domaće,2612
2944,206,Español,2332


In [26]:
# user count per track
df_playlist.groupby(["track", "artist"], as_index=False)["user_id"].nunique().sort_values(by=["user_id"], ascending=False).head(10)

,track,artist,user_id
104685,Little Talks,Of Monsters and Men,67
115417,Midnight City,M83,65
143602,Radioactive,Imagine Dragons,60
64551,Get Lucky - Radio Edit,Daft Punk,59
139207,Pompeii,Bastille,57
75834,Ho Hey,The Lumineers,55
22667,Blurred Lines,Robin Thicke,54
160655,Smells Like Teen Spirit,Nirvana,53
199260,Wake Me Up,Avicii,52
154985,Seven Nation Army,The White Stripes,51


In [27]:
# minimum artist and users entry number
df_playlist = df_playlist.groupby('artist').filter(lambda x: len(x)>=50)
df_playlist = df_playlist[df_playlist.groupby('user_id')["artist"].transform('nunique') >= 10]

In [28]:
size = lambda x: len(x)
#df_freq = df_playlist.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
#df_freq.head()

In [29]:
df_freq = df_playlist.groupby(['user_id', 'artist', 'track']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'track', 'freq']].sort_values(['freq'], ascending=False)
df_freq["artist-track"] = df_freq["artist"] + " - " + df_freq["track"]
df_freq

,user_id,artist,track,freq,artist-track
107587,239,The New Pornographers,Challengers,12,The New Pornographers - Challengers
139612,353,Yeah Yeah Yeahs,Dull Life,12,Yeah Yeah Yeahs - Dull Life
106048,239,Coldplay,Magic,12,Coldplay - Magic
178738,453,Yeah Yeah Yeahs,Maps,12,Yeah Yeah Yeahs - Maps
139629,353,Yeah Yeah Yeahs,Runaway,12,Yeah Yeah Yeahs - Runaway
...,...,...,...,...,...
67576,188,Audioslave,Exploder,1,Audioslave - Exploder
67577,188,Audioslave,Gasoline,1,Audioslave - Gasoline
67578,188,Audioslave,Getaway Car,1,Audioslave - Getaway Car
67579,188,Audioslave,Heaven's Dead,1,Audioslave - Heaven's Dead


In [30]:
df_artist_track = pd.DataFrame(df_freq["artist-track"].unique())
df_artist_track = df_artist_track.reset_index()
df_artist_track = df_artist_track.rename(columns={'index':'artist-track_id', 0: 'artist-track'})
df_artist_track.head()

,artist-track_id,artist-track
0,0,The New Pornographers - Challengers
1,1,Yeah Yeah Yeahs - Dull Life
2,2,Coldplay - Magic
3,3,Yeah Yeah Yeahs - Maps
4,4,Yeah Yeah Yeahs - Runaway


In [31]:
#df_artist = pd.DataFrame(df_freq["artist"].unique())
#df_artist = df_artist.reset_index()
#df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist'})
#df_artist.head()

In [32]:
df_freq = pd.merge(df_freq, df_artist_track, how='inner', on='artist-track')
df_freq.head()

,user_id,artist,track,freq,artist-track,artist-track_id
0,239,The New Pornographers,Challengers,12,The New Pornographers - Challengers,0
1,183,The New Pornographers,Challengers,3,The New Pornographers - Challengers,0
2,404,The New Pornographers,Challengers,2,The New Pornographers - Challengers,0
3,126,The New Pornographers,Challengers,1,The New Pornographers - Challengers,0
4,188,The New Pornographers,Challengers,1,The New Pornographers - Challengers,0


In [33]:
# https://github.com/aayushmnit/cookbook/blob/master/recsys.py
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions


In [34]:
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [35]:
def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [36]:
def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4):
    '''
    Function to run matrix-factorization algorithm
    Required Input -
        - interactions = dataset create by create_interaction_matrix
        - n_components = number of embeddings you want to create to define Item and user
        - loss = loss function other options are logistic, brp
        - epoch = number of epochs to run 
        - n_jobs = number of cores used for execution 
    Expected Output  -
        Model - Trained model
    '''
    
    #uncommented for train test split
    # x = sparse.csr_matrix(interactions.values)
    x = interactions
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

In [37]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items[:10]:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores[:10]:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [38]:
interactions = create_interaction_matrix(df = df_freq, user_col = "user_id", item_col = 'artist-track_id', rating_col = 'freq', norm= False, threshold = None)
interactions.head()

artist-track_id,0,1,2,3,4,5,6,7,8,9,...,96414,96415,96416,96417,96418,96419,96420,96421,96422,96423
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
user_dict = create_user_dict(interactions=interactions)

In [40]:
artists_track_dict = create_item_dict(df = df_artist_track, id_col = 'artist-track_id', name_col = 'artist-track')

In [41]:
x = sparse.csr_matrix(interactions.values)
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.2, random_state=None)

In [42]:
model = runMF(interactions = train,
                 n_components = 30,
                 loss = 'warp',
                 k = 15,
                 epoch = 30,
                 n_jobs = 1)

In [43]:
train_auc = auc_score(model, train, num_threads=4).mean()
print('Train AUC: %s' % train_auc)

Train AUC: 0.99862546


In [44]:
# avoid re-recomendations
test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
print('Test AUC: %s' % test_auc)

Test AUC: 0.7182925


In [45]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()
print('train Precision %.2f, test Precision %.2f.' % (train_precision, test_precision))

train Precision 0.62, test Precision 0.05.


In [46]:
rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = 3, 
                                      user_dict = user_dict,
                                      item_dict = artists_track_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Likes:
1- Demi Lovato - Heart By Heart
2- Colbie Caillat - When The Darkness Comes
3- All Time Low - Time-Bomb
4- All Time Low - Somewhere in Neverland
5- All Time Low - Something's Gotta Give
6- All Time Low - Sick Little Games
7- All Time Low - Poppin' - Video Mix
8- All Time Low - Lost In Stereo
9- All Time Low - Hello, Brooklyn
10- All Time Low - Break Your Little Heart

 Recommended Items:
1- Imagine Dragons - Radioactive
2- Imagine Dragons - It's Time
3- Ellie Goulding - Burn
4- Green Day - Holiday/Boulevard Of Broken Dreams
5- Carrie Underwood - Last Name
6- LMFAO - Party Rock Anthem
7- Jason Derulo - Talk Dirty (feat. 2 Chainz)
8- Bob Marley & The Wailers - Redemption Song
9- MIKA - We Are Golden
10- A Great Big World - Say Something


In [47]:
rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = 499, 
                                      user_dict = user_dict,
                                      item_dict = artists_track_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Likes:
1- Kelis - Good Stuff
2- Guillemots - Dancing In the Devils Shoes
3- The National - Abel
4- Suede - Snowblind
5- British Sea Power - Who’s In Control
6- George Michael - Fast Love
7- The Horrors - Sea Within A Sea
8- Noel Gallagher's High Flying Birds - AKA...What A Life!
9- Wild Beasts - Sweet Spot
10- The Antlers - I Don't Want Love

 Recommended Items:
1- Frank Ocean - Thinkin Bout You
2- Rihanna - We Found Love
3- Mogwai - How To Be A Werewolf
4- M.I.A. - Bad Girls
5- Diiv - How Long Have You Known
6- M.I.A. - Paper Planes
7- Azealia Banks - 212
8- Kanye West - Bound 2
9- Daft Punk - Get Lucky - Radio Edit
10- Florence + The Machine - You've Got The Love


In [48]:
import pickle
file_name = "lighfm.pkl"

# save
pickle.dump(model, open(file_name, "wb"))
#load
lightfm_model = pickle.load(open(file_name, "rb"))

# Unknown user

In [ ]:
#import surprise
#from sklearn.model_selection import train_test_split

In [50]:
top = df_freq.sort_values(['freq'], ascending=False).head(20)["artist-track"].values

In [51]:
top

array(['The New Pornographers - Challengers', 'Coldplay - Magic',
       'Yeah Yeah Yeahs - Dull Life', 'Yeah Yeah Yeahs - Maps',
       'Yeah Yeah Yeahs - Runaway', 'Wilco - Far, Far Away',
       'Band of Horses - Laredo',
       'The Flaming Lips - Yoshimi Battles The Pink Robots Part 1',
       'Yeah Yeah Yeahs - Shame and Fortune',
       'R.E.M. - Imitation Of Life',
       'The Postal Service - The District Sleeps Alone Tonight',
       'Death Cab for Cutie - Monday Morning',
       'The Head And The Heart - Another Story',
       'Death Cab for Cutie - No Room In Frame',
       'Yeah Yeah Yeahs - Dragon Queen', 'Broods - Bridges',
       'Wilco - You And I', 'Wilco - Misunderstood', 'Daughter - Youth',
       'R.E.M. - Man On The Moon'], dtype=object)